In [1]:
import tensorflow as tf
from tensorflow import keras as K
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.models import Model
from tensorflow.keras.layers import Input,Dense,Flatten,Softmax,Embedding,Reshape,SimpleRNN,GRU,LSTM,Bidirectional,Dropout
import numpy as np

tf.__version__, K.__version__

('1.9.0', '2.1.6-tf')

In [2]:
tf.reset_default_graph()
# S: Symbol that shows starting of decoding input
# E: Symbol that shows starting of decoding output
# P: Symbol that will fill in blank sequence if current batch data size is short than time steps

char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
num_dic = {n: i for i,n in enumerate(char_arr)}

seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]

# Seq2Seq Parameter
n_step = 5 # max lens of sequence
n_hidden = 128
n_class = len(num_dic) # number of class(=number of vocab)

def make_batch(seq_data):
    input_batch, output_batch, target_batch = [], [], []

    for seq in seq_data:
        for i in range(2):
            seq[i] = seq[i] + 'P' * (n_step - len(seq[i]))

        input_ = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(n_class)[input_])
        output_batch.append(np.eye(n_class)[output])

        target_batch.append(target)

    return np.array(input_batch), np.array(output_batch), np.array(target_batch).reshape(-1,6,1)

input_batch, output_batch, target_batch = make_batch(seq_data)

# Define Model
Reference:
- [](https://github.com/thecodacus/deep-learning-nmt-using-tf-keras/blob/master/seq2seq.py)

In [3]:
tf.reset_default_graph()
class Seq2Seq:
    def __init__(self, vocab_dim, decode_max_len, input_dim, output_dim, encoder_hidden):
    
        # create the model
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.encoder_hidden = encoder_hidden
        self.maxlen = decode_max_len
        self.vocab_dim = vocab_dim
        
        # Define an input sequence and process it.
        encoder_inputs = Input(shape=(self.input_dim, vocab_dim))
        encoder = LSTM(self.encoder_hidden, return_state=True)
        encoder_outputs, state_h, state_c = encoder(encoder_inputs)
        
        # We discard `encoder_outputs` and only keep states
        encoder_states = [state_h, state_c]
        
        # Set up the decoder, using `encoder_state` as initial state.
        # Decoder输入timestep = 最长序列长度+1(E标志)
        decoder_inputs = Input(shape=(self.output_dim+1, vocab_dim))
        # We set up our decoder to return full output sequences,
        # and to return internal states as well. We don't use the
        # return states in the training model, but we will use them in inference.
        decoder = LSTM(self.encoder_hidden, return_sequences=True, return_state=True)
        decoder_outputs,_,_ = decoder(decoder_inputs, initial_state = encoder_states)
        dense = Dense(vocab_dim, activation='softmax')
        decoder_outputs = dense(decoder_outputs)
        
        # Define the model that will turn
        # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
        self.trainingModel = Model([encoder_inputs,decoder_inputs], decoder_outputs)
        self.trainingModel.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
        
        # Inference setup
        self.encoder_model = Model(encoder_inputs, encoder_states) # Model(input,output)

        decoder_state_input_h = Input(shape=(self.encoder_hidden,))
        decoder_state_input_c = Input(shape=(self.encoder_hidden,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder_outputs, state_h, state_c = decoder(decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = dense(decoder_outputs)
        self.decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs])
        
    def trainModel(self, encoder_input_data, decoder_input_data, decoder_target_data, batch_size=6, epochs=500):
        self.trainingModel.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size,
                               epochs=epochs)
        self.trainingModel.save('s2s.h5')
    
    def loadWeights(self):
        self.trainingModel.load_weights('s2s.h5')
        
    def predict(self, inputs, outputs=None):
        encoder_states = self.encoder_model.predict(inputs)
        if not outputs:
            outputs = np.zeros((inputs.shape[0], self.output_dim+1, self.vocab_dim))
            outputs[:,:,num_dic['P']] = 1
            
        predictions = self.decoder_model.predict([outputs] + encoder_states)
        return [n for n in np.argmax(predictions, 2)]

In [4]:
model = Seq2Seq(n_class, n_step, n_step, n_step, n_hidden)

In [5]:
model.trainModel(input_batch, output_batch, target_batch)

Epoch 1/500
6/6 [==============================] - 1s 212ms/step - loss: 3.3688
Epoch 2/500
6/6 [==============================] - 0s 2ms/step - loss: 3.3218
Epoch 3/500
6/6 [==============================] - 0s 2ms/step - loss: 3.2805
Epoch 4/500
6/6 [==============================] - 0s 3ms/step - loss: 3.2299
Epoch 5/500
6/6 [==============================] - 0s 3ms/step - loss: 3.1566
Epoch 6/500
6/6 [==============================] - 0s 3ms/step - loss: 3.0368
Epoch 7/500
6/6 [==============================] - 0s 4ms/step - loss: 2.8322
Epoch 8/500
6/6 [==============================] - 0s 3ms/step - loss: 2.5559
Epoch 9/500
6/6 [==============================] - 0s 3ms/step - loss: 2.3630
Epoch 10/500
6/6 [==============================] - 0s 3ms/step - loss: 2.2716
Epoch 11/500
6/6 [==============================] - 0s 3ms/step - loss: 2.2029
Epoch 12/500
6/6 [==============================] - 0s 3ms/step - loss: 2.1437
Epoch 13/500
6/6 [==============================] - 0s 4ms/

Epoch 105/500
6/6 [==============================] - 0s 3ms/step - loss: 0.3250
Epoch 106/500
6/6 [==============================] - 0s 3ms/step - loss: 0.3176
Epoch 107/500
6/6 [==============================] - 0s 3ms/step - loss: 0.3140
Epoch 108/500
6/6 [==============================] - 0s 3ms/step - loss: 0.3148
Epoch 109/500
6/6 [==============================] - 0s 4ms/step - loss: 0.3242
Epoch 110/500
6/6 [==============================] - 0s 3ms/step - loss: 0.3235
Epoch 111/500
6/6 [==============================] - 0s 2ms/step - loss: 0.3183
Epoch 112/500
6/6 [==============================] - 0s 2ms/step - loss: 0.3232
Epoch 113/500
6/6 [==============================] - 0s 2ms/step - loss: 0.4387
Epoch 114/500
6/6 [==============================] - 0s 2ms/step - loss: 0.7184
Epoch 115/500
6/6 [==============================] - 0s 4ms/step - loss: 0.6667
Epoch 116/500
6/6 [==============================] - 0s 3ms/step - loss: 0.2689
Epoch 117/500
6/6 [=====================

Epoch 208/500
6/6 [==============================] - 0s 3ms/step - loss: 0.4658
Epoch 209/500
6/6 [==============================] - 0s 2ms/step - loss: 0.1636
Epoch 210/500
6/6 [==============================] - 0s 3ms/step - loss: 0.0375
Epoch 211/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0364
Epoch 212/500
6/6 [==============================] - 0s 3ms/step - loss: 0.0356
Epoch 213/500
6/6 [==============================] - 0s 3ms/step - loss: 0.0349
Epoch 214/500
6/6 [==============================] - 0s 3ms/step - loss: 0.0343
Epoch 215/500
6/6 [==============================] - 0s 3ms/step - loss: 0.0338
Epoch 216/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0333
Epoch 217/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0329
Epoch 218/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0324
Epoch 219/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0319
Epoch 220/500
6/6 [=====================

Epoch 311/500
6/6 [==============================] - 0s 3ms/step - loss: 0.0065
Epoch 312/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0064
Epoch 313/500
6/6 [==============================] - 0s 3ms/step - loss: 0.0064
Epoch 314/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0064
Epoch 315/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0066
Epoch 316/500
6/6 [==============================] - 0s 3ms/step - loss: 0.0070
Epoch 317/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0096
Epoch 318/500
6/6 [==============================] - 0s 3ms/step - loss: 0.0300
Epoch 319/500
6/6 [==============================] - 0s 2ms/step - loss: 0.3735
Epoch 320/500
6/6 [==============================] - 0s 3ms/step - loss: 0.2528
Epoch 321/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0544
Epoch 322/500
6/6 [==============================] - 0s 3ms/step - loss: 0.0084
Epoch 323/500
6/6 [=====================

Epoch 414/500
6/6 [==============================] - 0s 3ms/step - loss: 0.0018
Epoch 415/500
6/6 [==============================] - 0s 3ms/step - loss: 0.0017
Epoch 416/500
6/6 [==============================] - 0s 3ms/step - loss: 0.0017
Epoch 417/500
6/6 [==============================] - 0s 3ms/step - loss: 0.0017
Epoch 418/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 419/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 420/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 421/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0016
Epoch 422/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 423/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 424/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 425/500
6/6 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 426/500
6/6 [=====================

# 测试

In [13]:
result = model.predict(input_batch)

for i in range(len(input_batch)):
    decoded = [char_arr[i] for i in result[i]]
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    print(seq_data[i][0].replace('P',''), '->', translated.replace('P',''))

man -> women
black -> white
king -> queen
girl -> boy
up -> down
high -> low
